# Phenotype Synergy Analysis

This notebook contains code to interprete results from the synergy score analysis. 

In [1]:
import numpy as np
import pandas as pd
import datetime
import math
import os
import sys
import logging
mf_module_path = os.path.abspath(os.path.join('../python'))
if mf_module_path not in sys.path:
    sys.path.append(mf_module_path)
import mf
import mf_random
from ontology import Ontology
import pickle

In [23]:
# use hp.obo from online or your local file system
#hpo = Ontology('http://purl.obolibrary.org/obo/hp.obo')
hpo = Ontology('/Users/zhangx/git/human-phenotype-ontology/hp.obo')
hpo_term_map = hpo.term_id_2_label_map()

# base dir: path to the MIMIC_HPO repo on your machine
base_dir = '../../..'

# Mutual information without considering diagnosis

This section analyzes the mutual information between phenotype pairs (labHpo-labHpo, textHpo-labHpo, textHpo-textHpo) in regardless of diagnosis. 

Note that the same information is also produced as a side product when we calculate mutual information in respect to a disease. Because we run simulations in the latter case, we additionally get their p values for the observed value.

Processing steps: 
For each type of summary statistics, we can map them into a MutualInfoXY instance. From there, we can get the mutual information matrix (or dataframe), and then do necessary filtering. In the last, we save the dataframe to a file.

**Filter 1:**

If P1 and P2 are both from labHpo or textHpo, then remove rows where P1 is identical to P2

**Filter 2:**

If P1 and P2 are both from labHpo or textHpo, then a row for (a, b) is identical to (b, a). So one row is removed. 

**Filter 3:**

Because we automatically added ancestors terms if a child term is observed, P1 and P2 are expected to have high mutual information if they have dependency in HPO hierarchy. Such pairs are removed. 

In [75]:
def mf_dataframe_regardless_of_diagnosis(p1_source, p2_source, hpo_term_map):
    summary_file_name = 'summary_{}_{}.obj'.format(p1_source, p2_source)
    summary_file_path = os.path.join(base_dir, 'data', 'mf_regardless_of_diseases', summary_file_name)
    with open(summary_file_path, 'rb') as f:
        summary_statistics = pickle.load(f)
    
    # convert to a MutualInfoXY object from summary statistics
    mf_XY = mf.MutualInfoXY(summary_statistics)
    
    # get a dataframe
    df_mf_XY = mf_XY.mf_labeled()
    
    # label termid with names
    df_mf_XY['P1_label'] = np.array([hpo_term_map.get(termid) for termid in df_mf_XY.P1])
    df_mf_XY['P2_label'] = np.array([hpo_term_map.get(termid) for termid in df_mf_XY.P2])
    
    return df_mf_XY

def filter_mf_dataframe_regardless_of_diagnosis(df_mf_XY, hpo,
                                         remove_pairs_with_same_terms, 
                                         remove_reflective_pairs, 
                                         remove_pairs_with_dependency,
                                        sort_by='mf'):
    # remove pairs where P1, P2 are the same
    if remove_pairs_with_same_terms:
        df_mf_XY = df_mf_XY.loc[df_mf_XY.P1 != df_mf_XY.P2, :].reset_index(drop=True)
    
    # remove reflective pairs: (a, b) and (b, a) are considered reflective pairs
    # the method is to use string comparison: always require P1 <= P2
    if remove_reflective_pairs:
        df_mf_XY = df_mf_XY.loc[df_mf_XY.P1 <= df_mf_XY.P2, :].reset_index(drop=True)
    
    # remove pairs with dependency
    has_dependency = np.repeat(False, len(df_mf_XY))
    for i in np.arange(len(df_mf_XY)):
        x = df_mf_XY.P1[i]
        y = df_mf_XY.P2[i]
        has_dependency[i] = x != y and (hpo.exists_path(x, y) or hpo.exists_path(y, x))
        
    if remove_pairs_with_dependency:
        # when does two terms in a pair has dependency: different, but there is path from one to another
        df_mf_XY = df_mf_XY.loc[np.logical_not(has_dependency), :]
        
    df_mf_XY = df_mf_XY.sort_values(by=sort_by, ascending=False).reset_index(drop=True)
    
    return df_mf_XY

def save_mf_dataframe_regardless_of_diagnosis(df_mf_XY, p1_source, p2_source):
    # save to csv file
    output_file_name = 'mf_{}_{}.csv'.format(p1_source, p2_source)
    output_file_path = os.path.join(base_dir, 'data', 'mf_regardless_of_diseases', output_file_name)
    df_mf_XY.to_csv(output_file_path)
    
def process_mf_regardless_of_diagnosis(p1_source, p2_source, hpo, 
                            remove_pairs_with_same_terms, remove_reflective_pairs, remove_pairs_with_dependency):
    # step 1: make a labeled dataframe
    hpo_term_map = hpo.term_id_2_label_map()
    df_mf_XY = mf_dataframe_regardless_of_diagnosis(p1_source, p2_source, hpo_term_map)
    # step 2: filter unnecessary rows
    df_mf_XY = filter_mf_dataframe_regardless_of_diagnosis(df_mf_XY, hpo, remove_pairs_with_same_terms, 
                                                           remove_reflective_pairs, remove_pairs_with_dependency)
    # step 3: save to csv
    save_mf_dataframe_regardless_of_diagnosis(df_mf_XY, p1_source, p2_source)
    

### textHpo-labHpo

In [76]:
process_mf_regardless_of_diagnosis('textHpo', 'labHpo', hpo, 
                                   remove_pairs_with_same_terms=False, 
                                   remove_reflective_pairs=False, 
                                   remove_pairs_with_dependency=True)

### textHpo-textHpo

In [55]:
process_mf_regardless_of_diagnosis('textHpo', 'textHpo', hpo, 
                                   remove_pairs_with_same_terms=True, 
                                   remove_reflective_pairs=True, 
                                   remove_pairs_with_dependency=True)

### labHpo-labHpo

In [56]:
process_mf_regardless_of_diagnosis('labHpo', 'labHpo', hpo, 
                                   remove_pairs_with_same_terms=True, 
                                   remove_reflective_pairs=True, 
                                   remove_pairs_with_dependency=True)

<br>

## Mutual information between textHpo and labHpo in respect to diagnoses

At each admission, patients could receive multiple diagnosis codes. One of them is designated as "primary" (in MIMIC, it has a rank of 1) and others secondary (rank 2, 3...). Therefore, the analysis was run under two scenerios: 
1. Only primary diagnosis is considered. 
2. All diagnoses are considered equally. 

Under the first scenerio, a patient is considered to be a case only if the corresponding billing code is listed as "primary". While in the second case, a patient is considered to be a case when the corresponding billing code is listed as primary or secondary.   

In [95]:
def summary_statistics_to_mutualInfoXY_z(p1_source, p2_source, primary_only, diag_code):
    if primary_only:
        diag_dir = "primary_only"
    else:
        diag_dir = "primary_and_secondary"
        
    summaries_file_name = 'summaries_diagnosis_{}_{}.obj'.format(p1_source, p2_source)
    summaries_file_path = os.path.join(base_dir, 'data', 'mf_regarding_diseases', diag_dir, summaries_file_name)
    
    with open(summaries_file_path, 'rb') as f:
        summaries = pickle.load(f)
    
    summary_statistics_for_diag_code = summaries.get(diag_code)
    mutualInfoXYz = mf.MutualInfoXYz(summary_statistics_for_diag_code)
    
    return mutualInfoXYz

def mf_dataframes_regarding_diagnosis(mutualInfoXYz, **p_values):
    """
    @param p_values: output from simulation
    """
    assert isinstance(mutualInfoXYz, mf.MutualInfoXYz)
    # unpack p values
    p_mf_Xz = p_values.get('mf_Xz')
    p_mf_Yz = p_values.get('mf_Yz')
    p_mf_XY_z = p_values.get('mf_XY_z')
    p_mf_XY_given_z = p_values.get('mf_XY_given_z')
    p_synergy = p_values.get('synergy')
    p_mf_XY_omit_z = p_values.get('mf_XY_omit_z')
    
    X_labels, Y_labels = mutualInfoXYz.vars_labels.values()
    M1 = len(X_labels)
    M2 = len(Y_labels)

    mf_Xz = mutualInfoXYz.mutual_info_Xz()
    mf_Yz = mutualInfoXYz.mutual_info_Yz()

    # mutual information between single phenotypes and diagnosis
    df_mf_Xz = pd.DataFrame(data={'X': X_labels, 'mf_Xz': mf_Xz})
    df_mf_Yz = pd.DataFrame(data={'Y': Y_labels, 'mf_Yz': mf_Yz})
    # add p values
    df_mf_Xz['p_mf_Xz'] = p_mf_Xz if p_mf_Xz is not None else np.repeat(-1, M1)
    df_mf_Yz['p_mf_Yz'] = p_mf_Yz if p_mf_Yz is not None else np.repeat(-1, M2)
    
    # joint and conditional mutual information, and synergy
    mf_XY_z = mutualInfoXYz.mutual_info_XY_z()
    mf_XY_given_z = mutualInfoXYz.mutual_info_XY_given_z()
    mf_synergy = mutualInfoXYz.synergy_XY2z()
    
    # mutual information between phenotypes without considering diagnosis
    mf_XY_omit_z = mutualInfoXYz.mutual_info_XY_omit_z()
    
    # mutual information between phenotype pairs and diagnosis
    df_mf_XY_z = pd.DataFrame()
    df_mf_XY_z['X'] = np.repeat(X_labels, M2)
    df_mf_XY_z['Y'] = np.tile(Y_labels, [M1])
    df_mf_XY_z['mf_Xz'] = np.repeat(mf_Xz, M2)
    df_mf_XY_z['mf_Yz'] = np.tile(mf_Yz, [M1])
    df_mf_XY_z['mf_XY_z'] = mf_XY_z.flat
    df_mf_XY_z['synergy'] = mf_synergy.flat   # synergy = mf_XY_z - mf_Xz - mf_Yz
    
    # mutual information between phenotypes after omiting diagnosis or conditioned on diagnosis
    df_mf_XY_z['mf_XY_omit_z'] = mf_XY_omit_z.flat
    df_mf_XY_z['mf_XY_given_z'] = mf_XY_given_z.flat
    
    # ratio of conditional mutual information / mutual information without considering diagnosis
    # synergy is also equal to mf_XY_given_z - mf_XY_omit_z
    # here we use ratio, which can be considered as an alternative of the above definition of synergy
    df_mf_XY_z['mf_ratio'] = df_mf_XY_z['mf_XY_given_z'] / df_mf_XY_z['mf_XY_omit_z']
    
    # add p values; otherwise, assign -1
    df_mf_XY_z['p_mf_Xz'] = np.repeat(p_mf_Xz, M2) if p_mf_Xz is not None else np.repeat(-1, M1*M2)
    df_mf_XY_z['p_mf_Yz'] = np.tile(p_mf_Yz, [M1]) if p_mf_Yz is not None else np.repeat(-1, M1*M2)
    df_mf_XY_z['p_mf_XY_z'] = p_mf_XY_z.flat if p_mf_XY_z is not None else np.repeat(-1, M1*M2)
    df_mf_XY_z['p_synergy'] = p_synergy.flat if p_synergy is not None else np.repeat(-1, M1*M2)
    df_mf_XY_z['p_mf_XY_omit_z'] = p_mf_XY_omit_z.flat if p_mf_XY_omit_z is not None else np.repeat(-1, M1*M2)
    df_mf_XY_z['p_mf_XY_given_z'] = p_mf_XY_given_z.flat if p_mf_XY_given_z is not None else np.repeat(-1, M1*M2)
    
    
    # add raw counts: 8 additional columns
    joint_dist_keys = ['+++', '++-', '+-+', '+--', '-++', '-+-', '--+', '---']
    joint_dist_values = mutualInfoXYz.m2.reshape([-1, 8]).astype(int)
    raw_counts = {joint_dist_keys[i]: joint_dist_values[:,i] for i in np.arange(8)}
    df_mf_XY_z = df_mf_XY_z.assign(**raw_counts)
    df_mf_XY_z['sum'] = np.sum(joint_dist_values, axis=-1) # it's a constant for all rows
    
    return df_mf_Xz, df_mf_Yz, df_mf_XY_z


def rename_mf_dataframes(df_mf_Xz, df_mf_Yz, df_mf_XY_z):
    # map column names to more meaningful name for this context
    name_dict = {
        'X': 'P1', 
        'Y': 'P2', 
        'mf_Xz': 'mf_P1_diag',
        'mf_Yz': 'mf_P2_diag',
        'mf_XY_z': 'mf_P1P2_diag',
        'mf_XY_given_z': 'mf_P1P2_given_diag',
        'mf_XY_omit_z': 'mf_P1P2_omit_diag',
        'p_mf_XY_z': 'p_mf_P1P2_diag',
        'p_mf_XY_given_z': 'p_mf_P1P2_given_diag',
        'p_mf_XY_omit_z': 'p_mf_P1P2_omit_diag',
        'mf_Xz': 'mf_P1_diag', 
        'mf_Yz': 'mf_P2_diag',  
        'p_mf_Xz': 'p_mf_P1_diag', 
        'p_mf_Yz': 'p_mf_P2_diag'   
    }

    df_mf_Xz = df_mf_Xz.rename(columns=name_dict, errors='ignore') 
    df_mf_Yz = df_mf_Yz.rename(columns=name_dict, errors='ignore') 
    df_mf_XY_z = df_mf_XY_z.rename(columns=name_dict, errors='ignore')
    
    return df_mf_Xz, df_mf_Yz, df_mf_XY_z


def filter_mf_dataframe_regarding_diagnosis(df_mf_XY_z, hpo, 
                                            remove_pairs_with_same_terms, 
                                            remove_reflective_pairs, 
                                            remove_pairs_with_dependency,
                                           sort_by='synergy'):
    # use the same method as the one defined above, except to sort by a different column
    return filter_mf_dataframe_regardless_of_diagnosis(df_mf_XY_z, hpo,
                                         remove_pairs_with_same_terms, 
                                         remove_reflective_pairs, 
                                         remove_pairs_with_dependency,
                                         sort_by)

def entropy(case, control):
    total = case + control 
    h = -(case / total * np.log2(case/total) + control/total * np.log2(control/total))
    return h
    

def load_p_values(p1_source, p2_source, diag_code, primary_only):
    if primary_only:
        p_values_file_name = 'p_value_{}_{}_{}_{}.obj'.format(p1_source, p2_source, diag_code, 'primary_only')
    else:
        p_values_file_name = 'p_value_{}_{}_{}_{}.obj'.format(p1_source, p2_source, diag_code, 'primary_and_secondary')
    
    p_values_file_path = os.path.join(base_dir, 'data', 'mf_regarding_diseases', 'primary_only', diag_code, p_values_file_name)
    with open(p_values_file_path, 'rb') as f:
        p = pickle.load(f)
    return p

convert_to_percent = np.vectorize(lambda x: ' {:.2f}%'.format(x * 100))

In [106]:
def process_mf_df_regarding_diseases(p1_source, p2_source, primary_only, diag_code, hpo, 
                                     remove_pairs_with_same_terms,
                                     remove_reflective_pairs, 
                                     remove_pairs_with_dependency,
                                     sort_by='synergy'):
    # calculate mutual information from summary statistics 
    mutualInfoXYz = summary_statistics_to_mutualInfoXY_z(p1_source, p2_source, primary_only, diag_code=disease)
    # load p values (calculated from simulation on Helix)
    p_values = load_p_values(p1_source, p2_source, disease, primary_only)
    # create dataframes
    df_mf_Xz, df_mf_Yz, df_mf_XY_z = mf_dataframes_regarding_diagnosis(mutualInfoXYz, **p_values)
    # rename dataframes 
    df_mf_Xz, df_mf_Yz, df_mf_XY_z = rename_mf_dataframes(df_mf_Xz, df_mf_Yz, df_mf_XY_z)
    # label HPO term ids with their names
    df_mf_Xz['P1_label'] = [hpo_term_map.get(termid) for termid in df_mf_Xz.P1]
    df_mf_Yz['P2_label'] = [hpo_term_map.get(termid) for termid in df_mf_Yz.P2]
    df_mf_XY_z['P1_label'] = [hpo_term_map.get(termid) for termid in df_mf_XY_z.P1]
    df_mf_XY_z['P2_label'] = [hpo_term_map.get(termid) for termid in df_mf_XY_z.P2]
    # filter synergy dataframe
    df_mf_XY_z = filter_mf_dataframe_regarding_diagnosis(df_mf_XY_z, hpo, 
                                                               remove_pairs_with_same_terms, 
                                                               remove_reflective_pairs, 
                                                               remove_pairs_with_dependency,
                                                               sort_by)
    # sort by desired columns
    df_mf_Xz = df_mf_Xz.sort_values(by='mf_P1_diag', ascending=False).reset_index(drop=True)
    df_mf_Yz = df_mf_Yz.sort_values(by='mf_P2_diag', ascending=False).reset_index(drop=True)
    df_mf_XY_z = df_mf_XY_z.sort_values(by=sort_by, ascending=False).reset_index(drop=True)
    
    # output to csv file
    # just save df_mf_XY_z_filtered as it contains data in df_mf_Xz and df_mf_Yz
    csv_file_name = 'df_synergy_{}_{}_{}.csv'.format(p1_source, p2_source, disease)
    if primary_only:
        diag_dir = 'primary_only'
    else:
        diag_dir = 'primary_and_secondary'    
    csv_parent_dir = os.path.join(base_dir, 'data', 'mf_regarding_diseases', diag_dir)
    if not os.path.exists(csv_parent_dir):
        os.mkdir(csv_parent_dir)
    csv_file_path = os.path.join(csv_parent_dir, csv_file_name)
    
    df_mf_XY_z.to_csv(csv_file_path)
    
    # output cytoscape files
    percentile = 0.01
    n = math.floor(len(df_mf_XY_z) * percentile)

    df_4_cytoscape = df_mf_XY_z \
        .assign(P1 = lambda x: 'Rad_' + x['P1']) \
        .assign(P2 = lambda x: 'Lab_' + x['P2']) \
        .head(n = n)

    cytoscape_dir = os.path.join(base_dir, 'data', 'mf_regarding_diseases', diag_dir, disease, 'cytoscape')
    if not os.path.exists(cytoscape_dir):
        os.mkdir(cytoscape_dir)
    # edges
    edges_path = os.path.join(cytoscape_dir, 'edges_{}_{}_{}.csv'.format(p1_source, p2_source, disease))
    df_4_cytoscape.loc[:, ['P1', 'P2', 'synergy', 'p_synergy']].to_csv(edges_path)

    # nodes
    nodes = pd.DataFrame(data={'term_id': np.concatenate([df_4_cytoscape.P1, df_4_cytoscape.P2]), 
                               'term_label': np.concatenate([df_4_cytoscape.P1_label, df_4_cytoscape.P2_label]),
                              'type': np.repeat(['Rad', 'Lab'], len(df_4_cytoscape))}).drop_duplicates()
    nodes_path = os.path.join(cytoscape_dir, 'nodes_textHpo_labHpo_{}.csv'.format( disease))
    nodes.to_csv(nodes_path)
    
    

In [107]:
p1_source='textHpo'
p2_source='labHpo'
primary_only=True
diag_code='493' 
remove_pairs_with_same_terms=False
remove_reflective_pairs=False
remove_pairs_with_dependency=True

process_mf_df_regarding_diseases(p1_source, p2_source, primary_only, diag_code, hpo, 
                                     remove_pairs_with_same_terms,
                                     remove_reflective_pairs, 
                                     remove_pairs_with_dependency,
                                     sort_by='synergy')

In [96]:
p1_source='textHpo'
p2_source='labHpo'
disease='493'
mutualInfoXYz = summary_statistics_to_mutualInfoXY_z(p1_source, p2_source, primary_only=True, diag_code=disease)

p_values = load_p_values(p1_source, p2_source, disease, primary_only=True)

df_mf_Xz, df_mf_Yz, df_mf_XY_z = mf_dataframes_regarding_diagnosis(mutualInfoXYz, **p_values)

df_mf_Xz, df_mf_Yz, df_mf_XY_z = rename_mf_dataframes(df_mf_Xz, df_mf_Yz, df_mf_XY_z)

df_XY_z_filtered = filter_mf_dataframe_regarding_diagnosis(df_mf_XY_z, hpo, 
                                            remove_pairs_with_same_terms=False, 
                                            remove_reflective_pairs=False, 
                                            remove_pairs_with_dependency=True)
df_XY_z_filtered.head()

,P1,P2,mf_P1_diag,mf_P2_diag,mf_P1P2_diag,synergy,mf_P1P2_omit_diag,mf_P1P2_given_diag,mf_ratio,p_mf_P1_diag,...,p_mf_P1P2_given_diag,+++,++-,+-+,+--,-++,-+-,--+,---,sum
0,HP:0002098,HP:0004364,0.000842,0.000308,0.001615,0.000466,0.022870,0.023336,1.020365,0.0,...,0.0,28,6872,58,2710,46,23473,82,25707,58976
1,HP:0002098,HP:0002157,0.000842,0.000178,0.001463,0.000444,0.023635,0.024078,1.018772,0.0,...,0.0,25,6338,61,3244,45,20425,83,28755,58976
2,HP:0002098,HP:0003138,0.000842,0.000134,0.001398,0.000422,0.023997,0.024419,1.017581,0.0,...,0.0,25,6185,61,3397,45,19553,83,29627,58976
3,HP:0002098,HP:0010929,0.000842,0.001223,0.002477,0.000412,0.011807,0.012220,1.034901,0.0,...,0.0,33,7717,53,1865,39,31765,89,17415,58976
4,HP:0002098,HP:0031970,0.000842,0.000180,0.001429,0.000408,0.023358,0.023766,1.017466,0.0,...,0.0,27,6424,59,3158,45,20951,83,28229,58976


In [46]:
primary_only = True
if primary_only:
    diag_dir = "primary_only"
else:
    diag_dir = "primary_and_secondary"

    
with open('../../../data/mf_regarding_diseases/{}/summaries_diagnosis_textHpo_labHpo.obj'.format(diag_dir), 'rb') as f:
    summaries_diagnosis_textHpo_labHpo = pickle.load(f)
with open('../../../data/mf_regarding_diseases/{}/summaries_diagnosis_textHpo_textHpo.obj'.format(diag_dir), 'rb') as f:
    summaries_diagnosis_textHpo_textHpo = pickle.load(f)
with open('../../../data/mf_regarding_diseases/{}/summaries_diagnosis_labHpo_labHpo.obj'.format(diag_dir), 'rb') as f:
    summaries_diagnosis_labHpo_labHpo = pickle.load(f)
summaries = {('textHpo', 'labHpo'): summaries_diagnosis_textHpo_labHpo,
             ('textHpo', 'textHpo'): summaries_diagnosis_textHpo_textHpo,
            ('labHpo', 'labHpo'): summaries_diagnosis_labHpo_labHpo}

In [47]:
# map column names to more meaningful name for this context
name_dict = {
        'X': 'P1', 
        'Y': 'P2', 
        'mf_Xz': 'mf_P1_diag',
        'mf_Yz': 'mf_P2_diag',
        'mf_XY_z': 'mf_P1P2_diag',
        'mf_XY_given_z': 'mf_P1P2_given_diag',
        'mf_XY_omit_z': 'mf_P1P2_omit_diag',
        'p_mf_XY_z': 'p_mf_P1P2_diag',
        'p_mf_XY_given_z': 'p_mf_P1P2_given_diag',
        'p_mf_XY_omit_z': 'p_mf_P1P2_omit_diag',
        'mf_Xz': 'mf_P1_diag', 
        'mf_Yz': 'mf_P2_diag',  
        'p_mf_Xz': 'p_mf_P1_diag', 
        'p_mf_Yz': 'p_mf_P2_diag'   
    }

### Output rendered csv and html files for selected diseases and phenotype sources

In [49]:
# change the following variables to analyze
disease = '493'
p1_source = 'textHpo'
p2_source = 'labHpo'


# calculate mutual information from summary statistics
summaries_p1_p2 = summaries.get((p1_source, p2_source))
summary_statistics = summaries_p1_p2[disease]
mf_diagnosis_phenotypes = mf.MutualInfoXYz(summary_statistics)

# calculate p value on Helix for disease of interest: input-summary statistics; output-a dictionary of p values
# TODO: get script 

# load p values
p_values_file_name = 'p_value_{}_{}_{}_{}.obj'.format(p1_source, p2_source, disease, diag_dir)
p_values_file_path = os.path.join('../../../data/mf_regarding_diseases/', diag_dir, disease, p_values_file_name)
p_values = load_p_values(p_values_file_path)

df_mf_Xz, df_mf_Yz, df_mf_XY_z = mf_dataframes(mf_diagnosis_phenotypes, **p_values)

df_mf_XY_z_filtered = filter_df(df_mf_Xz, df_mf_Yz, df_mf_XY_z)

# rename column names for this context using the dictionary defined above
df_mf_Xz = df_mf_Xz.rename(columns=name_dict, errors='ignore') 
df_mf_Yz = df_mf_Yz.rename(columns=name_dict, errors='ignore') 
df_mf_XY_z_filtered = df_mf_XY_z_filtered.rename(columns=name_dict, errors='ignore')

# label HPO term ids with their names
df_mf_Xz['P1_label'] = [hpo_term_map.get(termid) for termid in df_mf_Xz.P1]
df_mf_Yz['P2_label'] = [hpo_term_map.get(termid) for termid in df_mf_Yz.P2]
df_mf_XY_z_filtered['P1_label'] = [hpo_term_map.get(termid) for termid in df_mf_XY_z_filtered.P1]
df_mf_XY_z_filtered['P2_label'] = [hpo_term_map.get(termid) for termid in df_mf_XY_z_filtered.P2]

# sort by desired columns
df_mf_Xz = df_mf_Xz.sort_values(by='mf_P1_diag', ascending=False).reset_index(drop=True)
df_mf_Yz = df_mf_Yz.sort_values(by='mf_P2_diag', ascending=False).reset_index(drop=True)
df_mf_XY_z_filtered = df_mf_XY_z_filtered.sort_values(by='synergy', ascending=False).reset_index(drop=True)

# output to csv file
# just save df_mf_XY_z_filtered as it contains data in df_mf_Xz and df_mf_Yz
csv_file_name = 'df_synergy_{}_{}_{}.csv'.format(p1_source, p2_source, disease)
csv_file_path = os.path.join('../../../data/mf_regarding_diseases/', diag_dir, csv_file_name)
df_mf_XY_z_filtered.head(n = 20)
df_mf_XY_z_filtered.to_csv(csv_file_path)

# render html


# output cytoscape files
percentile = 0.01
n = math.floor(len(df_mf_XY_z_filtered) * percentile)

df_4_cytoscape = df_mf_XY_z_filtered \
    .assign(P1 = lambda x: 'Rad_' + x['P1']) \
    .assign(P2 = lambda x: 'Lab_' + x['P2']) \
    .head(n = n)


# edges
edges_path = os.path.join('../../../data/mf_regarding_diseases', diag_dir, 'cytoscape', 'edges_{}_{}_{}.csv'.format(p1_source, p2_source, disease))
df_4_cytoscape.loc[:, ['P1', 'P2', 'synergy', 'p_synergy']].to_csv(edges_path)

# nodes
nodes = pd.DataFrame(data={'term_id': np.concatenate([df_4_cytoscape.P1, df_4_cytoscape.P2]), 
                           'term_label': np.concatenate([df_4_cytoscape.P1_label, df_4_cytoscape.P2_label]),
                          'type': np.repeat(['Rad', 'Lab'], len(df_4_cytoscape))}).drop_duplicates()
nodes_path = os.path.join('../../../data/mf_regarding_diseases', diag_dir, 'cytoscape', 'nodes_textHpo_labHpo_{}.csv'.format( disease))
nodes.to_csv(nodes_path)
